In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
keywordSet = set()
citiesSet = set()
countriesSet = set()
locationsSet = set()
arrayWithMinorKeywords = set()
arrayWithForbbidenKeys = ['news', 'mass']

In [ ]:
def CorrectString(setWithPossibleKeywords, text, considerHashtag):
    word = 'Unknown'
    for each in setWithPossibleKeywords:
        if (re.search('(^|\s|#)' + re.escape(each) + '(?!\S)' , text, re.IGNORECASE)):
            word = each;
            break
    if (considerHashtag and word == 'Unknown'):
        for each in text.split():
            if (each.startswith('#')):
                word = each.lstrip('#')
    return word.lower()

def isForbidden(string):
    for word in arrayWithForbbidenKeys:
        if word in string.lower():
            return True
    return False

In [ ]:
def fixMissingFields(arrayWithSets, row, nameOfColumnWithParamMissing = 'keyword', nameOfPlaceWithText = 'text', useHashtag = True): 
    if (isinstance(row[nameOfColumnWithParamMissing], str)):
        row[nameOfColumnWithParamMissing] = row[nameOfColumnWithParamMissing].lower().rstrip().lstrip().replace('%20', ' ')
        return row
    for i in range(len(arrayWithSets)):
        startWritingUnknown = False
        startUsingHashtag = False
        if (i == len(arrayWithSets) - 1):
            startWritingUnknown = True
            startUsingHashtag = useHashtag
        wordToPlace = CorrectString(arrayWithSets[i], row[nameOfPlaceWithText], startUsingHashtag)
        if (wordToPlace != 'unknown' or startWritingUnknown):
            row[nameOfColumnWithParamMissing] = wordToPlace
            break
    return row


In [ ]:
def addToSet(nameOfSet, dataFrameRows, shouldCheckForbidden = False):
    for each in dataFrameRows:
        if (each == '' or each.isspace() or (shouldCheckForbidden and isForbidden(each))):
            continue
        nameOfSet.add(each.lower().replace('%20', ' ').lstrip().rstrip())

In [ ]:
#Define the sets and texts
tweets = pd.read_csv('train.csv')
dfWithKeywords = tweets.dropna(subset=['keyword'])
dfWithLocations = tweets.dropna(subset=['location'])
locations = pd.read_csv('worldcities.csv', encoding = 'latin-1')
addToSet(keywordSet, dfWithKeywords['keyword'])
addToSet(locationsSet, dfWithLocations['location'], True)
addToSet(citiesSet, locations['city'])
addToSet(countriesSet, locations['country'])
arrayWithSets = [locationsSet, citiesSet, countriesSet]
locationsSet.remove('on')
locationsSet.remove('in')
l =[]
for each in locationsSet:
    if (len(each.strip('?')) < 2):
        l.append(each)
for each in l:
    locationsSet.remove(each)
for value in keywordSet:
    for key in keywordSet:
        if (value == key):
            continue
        if value in key:
            arrayWithMinorKeywords.add(value)
            break
for value in arrayWithMinorKeywords:
    keywordSet.remove(value)
    

In [ ]:
#Fix missing fields, the second part WILL take a long time, there is no going around that, maybe using vectorize would improve it, but is a major unnecesary refactor
tweets = tweets.apply(lambda x: fixMissingFields([keywordSet, arrayWithMinorKeywords], x, useHashtag = False), axis = 1)
tweets = tweets.apply(lambda x: fixMissingFields(arrayWithSets, x, 'location', 'text', False), axis = 1)

In [ ]:
tweets['location'] = tweets['location'].apply(lambda x: x.lower().rstrip().lstrip().strip(','))
tweets.to_csv('withoutEncoding.csv', index = False) #Used to save a csv with that name at the folder where this file is